In [1]:
!conda info --envs

# conda environments:
#
base                     /home/tnguyen/miniconda
coref                 *  /home/tnguyen/miniconda/envs/coref
corefs                   /home/tnguyen/miniconda/envs/corefs
experiment               /home/tnguyen/miniconda/envs/experiment
fairseq                  /home/tnguyen/miniconda/envs/fairseq
ma_dialogue              /home/tnguyen/miniconda/envs/ma_dialogue
seq                      /home/tnguyen/miniconda/envs/seq



In [1]:
from fairseq.models.bart import BARTModel
import torch
from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [2]:
bart = BARTModel.from_pretrained(
    'checkpoints_multi_view',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='./cnn_dm-bin_2'
)

here!


In [3]:
bart.eval()
#bart.cpu()
bart.cuda()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [4]:
from tqdm import tqdm as tqdm
import torch

count = 1
bsz = 8
test_trans = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.source'
test_c99 = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_c99_label.source'
test_hypo = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best.hypo'
with open(test_trans) as source, open(test_c99) as source2, open(test_hypo, 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]

    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
                torch.cuda.empty_cache()
            
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:

        hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
       

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()

        

  0%|          | 0/499 [00:00<?, ?it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/search.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.beams_buf = self.indices_buf // vocab_size
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [2].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered inte

/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/search.py:79: UserWarning: An output with one or more elements was resized since it had shape [6, 8], which does not match the required output shape [5, 8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=(self.scores_buf, self.indices_buf),
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [3], which does not match the required output shape [0].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Trigg

/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:559: UserWarning: An output with one or more elements was resized since it had shape [2, 8], which does not match the required output shape [1, 8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=active_mask,
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:567: UserWarning: An output with one or more elements was resized since it had shape [2, 4], which does not match the required output shape [1, 4].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered 

  6%|▋         | 32/499 [00:14<03:28,  2.24it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [4].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=eos_bbsz_idx,
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [4], which does not match the required output shape [1].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero el

 14%|█▍        | 72/499 [00:35<03:35,  1.98it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:559: UserWarning: An output with one or more elements was resized since it had shape [4, 8], which does not match the required output shape [2, 8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=active_mask,
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:567: UserWarning: An output with one or more elements was resized since it had shape [4, 4], which does not match the required output shape [2, 4].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace,

 24%|██▍       | 120/499 [01:01<03:14,  1.95it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:505: UserWarning: An output with one or more elements was resized since it had shape [5], which does not match the required output shape [1].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=eos_scores,
 37%|███▋      | 184/499 [01:36<02:55,  1.79it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:505: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [6].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an 

 69%|██████▉   | 344/499 [02:59<01:12,  2.13it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [5], which does not match the required output shape [1].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=eos_bbsz_idx,
100%|██████████| 499/499 [04:31<00:00,  1.84it/s]


In [5]:
from rouge import Rouge, FilesRouge
hyp_path = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best.hypo'
ref_path = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.target'
hypothesis = []
with open(hyp_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        hypothesis.append(l[:-1])

reference = []
with open(ref_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        reference.append(l[:-1])

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference, avg = True)
#print('Test', rouge.get_scores(hypothesis, reference, avg = True))
            
print('TEST')
for key, metric in scores.items():
    print(key)
    for k, v in metric.items():
        print(f'{k}: {v}')

TEST
rouge-1
r: 0.42381762924534044
p: 0.3106988218727275
f: 0.34378587073025224
rouge-2
r: 0.16600478963012985
p: 0.12211266858958905
f: 0.13351775470396782
rouge-l
r: 0.3963902365567834
p: 0.2902285889817407
f: 0.3213212855506028


In [6]:
for i in range(5):
    print('PRED:')
    print(hypothesis[i])
    print('LABEL:')
    print(reference[i])
    print('\n')

PRED:
Person1# sends an intra-office memorandum to all employees. The use of Instant Message programs by employees during working hours is strictly prohibited. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation. At second offense, the employee will face termination. The policy applies to internal and external communications.
LABEL:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


PRED:
Person2# got stuck in a traffic jam near the Carrefour intersection. She will consider taking public transport system to work instead of driving to work.
LABEL:
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use public transportations to keep healthy and to protect the environment.


PRED:
Masha and Hero are getting divorced. They filed for a divorce. The divorce will be final in the New Year.
LABEL:

In [7]:
bart = BARTModel.from_pretrained(
    'train_sh/checkpoints_stage',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='../dialogsum-bin'
)

bart.eval()
bart.cuda()

here!


BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEnc

In [8]:
from tqdm import tqdm as tqdm
import torch

count = 1
bsz = 8
test_trans = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.source'
test_c99 = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_c99_label.source'
test_hypo = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best.hypo'

with open(test_trans) as source, open(test_c99) as source2, open(test_hypo, 'wt', encoding='utf-8') as fout:
    s1 = source.readlines()
    s2 = source2.readlines()

    slines = [s1[0].strip()]
    slines2 = [s2[0].strip()]

    for i in tqdm(range(1, len(s1))):
        if count % bsz == 0:
            with torch.no_grad():
                hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
                torch.cuda.empty_cache()
            
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + '\n')
                fout.flush()
            slines = []
            slines2 = []

        slines.append(s1[i].strip())
        slines2.append(s2[i].strip())

        count += 1

    if slines != []:

        hypotheses_batch = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
       

        for hypothesis in hypotheses_batch:
            fout.write(hypothesis + '\n')
            fout.flush()


  0%|          | 0/499 [00:00<?, ?it/s]/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:497: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [6].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  out=eos_bbsz_idx,
/home/tnguyen/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/sequence_generator.py:559: UserWarning: An output with one or more elements was resized since it had shape [8, 8], which does not match the required output shape [5, 8].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero eleme

100%|██████████| 499/499 [05:07<00:00,  1.62it/s]


In [9]:
from rouge import Rouge, FilesRouge

hyp_path = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best.hypo'
ref_path = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.target'
hypothesis = []
with open(hyp_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        hypothesis.append(l[:-1])

reference = []
with open(ref_path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        reference.append(l[:-1])

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference, avg = True)
#print('Test', rouge.get_scores(hypothesis, reference, avg = True))
            
print('TEST')
for key, metric in scores.items():
    print(key)
    for k, v in metric.items():
        print(f'{k}: {v}')

TEST
rouge-1
r: 0.5678239788779302
p: 0.3457162632182704
f: 0.4214851315520334
rouge-2
r: 0.24301067687032568
p: 0.13907783503136428
f: 0.1724773875682337
rouge-l
r: 0.5112094712565164
p: 0.31107723812728544
f: 0.3794061540863322


In [10]:
for i in range(5):
    print('PRED:')
    print(hypothesis[i])
    print('LABEL:')
    print(reference[i])
    print('\n')

PRED:
#Person1# asks Ms. Dawson to take a dictation for him and asks her to send an intra-office memorandum to all employees by this afternoon. He tells her that Instant Message programs by employees during working hours are strictly prohibited and applies to internal and external communications. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation, and at second offense, the employee will face termination.
LABEL:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


PRED:
#Person2# got stuck in traffic again. #Person1# suggests taking public transport system to work to get home and suggests biking to work when it's nicer outside, but #Person2 # is going to miss having the freedom with a car. Then they talk about the environment.
LABEL:
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use p